### Import of data

In [188]:
import requests
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

### Data we going to use in csv

In [189]:
features = ["hktv_ad_id", "order_value", "primary_category_name_en"]

### Import csv, from 2021 November to 2022 November, each months 1st day

In [190]:
csv_1 = pd.read_csv('csv/Public_Transaction_2021-11-01.csv', usecols=features)
csv_2 = pd.read_csv('csv/Public_Transaction_2021-12-01.csv', usecols=features)
csv_3 = pd.read_csv('csv/Public_Transaction_2022-01-01.csv', usecols=features)
csv_4 = pd.read_csv('csv/Public_Transaction_2022-02-01.csv', usecols=features)
csv_5 = pd.read_csv('csv/Public_Transaction_2022-03-01.csv', usecols=features)
csv_6 = pd.read_csv('csv/Public_Transaction_2022-04-01.csv', usecols=features)
csv_7 = pd.read_csv('csv/Public_Transaction_2022-05-01.csv', usecols=features)
csv_8 = pd.read_csv('csv/Public_Transaction_2022-06-01.csv', usecols=features)
csv_9 = pd.read_csv('csv/Public_Transaction_2022-07-01.csv', usecols=features)
csv_10 = pd.read_csv('csv/Public_Transaction_2022-08-01.csv', usecols=features)
csv_11 = pd.read_csv('csv/Public_Transaction_2022-09-01.csv', usecols=features)
csv_12 = pd.read_csv('csv/Public_Transaction_2022-10-01.csv', usecols=features)
csv_13 = pd.read_csv('csv/Public_Transaction_2022-11-01.csv', usecols=features)

### Joining all the csv into a large dataframe

In [191]:
df_1 = pd.concat([csv_1,csv_2,csv_3,csv_4,csv_5,csv_6,csv_7,csv_8,csv_9,csv_10,csv_11,csv_12,csv_13], ignore_index=True)

### Number of Data

In [192]:
df_1.shape[0]

3030730

### Read the user's data including gender and age range

In [193]:
f = open("sampleJSON.txt", encoding='utf-8').read()
feature = ["hktv_ad_id", "gender", "age_range"]
t = f.split("\n")
t.pop()
empty = []
for i in t:
    rows = json.loads(i)
    empty.append([rows["hktv_ad_id"], rows["gender"], rows["age_range"]])
df_2 = pd.DataFrame(empty, columns=feature)

### order value : From range (i.e. 5000-6000) to value (5000+6000 /2 = 5500)

In [194]:
#Make grouped data to numerical data

data = []
for i in range(len(df_1['order_value'])):
    num = df_1['order_value'][i].split(' - ')
    data.append((int(num[0])+int(num[1]))/2)

df_1["price"] = data

### same as order value, from age_range ot age

In [195]:
#Make grouped data to numerical data

data = []
for i in range(len(df_2['age_range'])):
    if df_2['age_range'][i] == None or df_2['age_range'][i] == "":
        data.append(np.nan)
        continue
    if df_2['age_range'][i].split('_')[1] == '55plus':
        data.append(60)
    elif df_2['age_range'][i].split('_')[1] == "12":
        data.append(10)
    else:
        num = df_2['age_range'][i].split('_')
        data.append((int(num[1])+int(num[2]))/2)

df_2['age'] = data

### Merging the transaction table and user data table, by the customer id (hktv_ad_id)

In [196]:
df = pd.merge(df_1, df_2, on="hktv_ad_id")
df.drop(labels=["order_value", "age_range"], axis=1, inplace=True)

In [197]:
#dealing with null and none data?

#price -> mean according to?
df['price'].fillna(value = df['price'].mean(), inplace = True)
#age -> mean according to?
df['age'].fillna(value = df['age'].mean(), inplace = True)
#gender = none
df.dropna(inplace=True)

In [198]:
#Check for the missing value again
df.isna().sum()

hktv_ad_id                  0
primary_category_name_en    0
price                       0
gender                      0
age                         0
dtype: int64

In [221]:
#Rename the 'primary_category_name_en' to 'category'
df.rename(columns={'primary_category_name_en':'category', 'hktv_ad_id':'user'}, inplace=True)
#Reorder to let the target value to be the last column
df = df.reindex(columns=["user", "gender", "age", "category", "price"])

In [222]:
from sklearn.preprocessing import LabelEncoder

In [223]:
l1 = LabelEncoder()
cat = ['category', 'gender']
for parameter in cat:
    l1.fit(df[parameter])
    df[parameter] = l1.transform(df[parameter])

In [224]:
# Number of data after merge and concating all csv
df.shape[0]

106446

In [238]:
df.head()

,user,gender,age,category,price
8,LHXRgMIvLsJIsBZ9EcE6h8jLWkWboqseR1ftfkTPD/U=,0,22.5,12,324.5
17,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
18,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
19,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5
20,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5


# Dimention Reduction JT <hr>

In [225]:
df.head()

,user,gender,age,category,price
8,LHXRgMIvLsJIsBZ9EcE6h8jLWkWboqseR1ftfkTPD/U=,0,22.5,12,324.5
17,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
18,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
19,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5
20,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5


In [226]:
from sklearn.preprocessing import StandardScaler

In [227]:
variables = ['category', 'gender', 'age']
x = df.loc[:, variables].values
y = df.loc[:,['price']].values
x = StandardScaler().fit_transform(x)
x = pd.DataFrame(x)

In [228]:
from sklearn.decomposition import PCA

In [229]:
pca = PCA()
x_pca = pca.fit_transform(x)
x_pca = pd.DataFrame(x_pca)
x_pca.head()

,0,1,2
0,-1.550585,-0.286133,-0.960722
1,-0.851856,-0.547489,-0.397440
2,-0.851856,-0.547489,-0.397440
3,-0.851856,-0.547489,-0.397440
4,-0.851856,-0.547489,-0.397440


### From Below, we can see that only two parameter cannot even capture 70% of the variance of the data, therefore we won't conduct any dimention reduction here.

In [230]:
explained_variance = pca.explained_variance_ratio_
explained_variance[:2].sum()

0.6851997741887051

In [231]:
x_pca['price']=y
x_pca.columns = ['category', 'gender', 'age', 'price']
x_pca.head()

,category,gender,age,price
0,-1.550585,-0.286133,-0.960722,324.5
1,-0.851856,-0.547489,-0.397440,424.5
2,-0.851856,-0.547489,-0.397440,424.5
3,-0.851856,-0.547489,-0.397440,24.5
4,-0.851856,-0.547489,-0.397440,24.5


# Regression Nathalie<hr>

## Thing's to do <br>
<h3><ol>
<li> train test split: Use sklearn library, 80% training 20% test</li>
<li> Model Training: use the best degree aka the best fit line</li>
<li> Performance Testing: Use R2 and RMS From HW 5</li>
</ol></h3>

In [236]:
df.head()

,user,gender,age,category,price
8,LHXRgMIvLsJIsBZ9EcE6h8jLWkWboqseR1ftfkTPD/U=,0,22.5,12,324.5
17,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
18,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
19,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5
20,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5


# Decision Tree Joseph<hr>

In [233]:
df.head()

,user,gender,age,category,price
8,LHXRgMIvLsJIsBZ9EcE6h8jLWkWboqseR1ftfkTPD/U=,0,22.5,12,324.5
17,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
18,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
19,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5
20,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5


## Thing's to do <br>
<h3><ol>
<li> train test split: Use sklearn library, 80% training 20% test</li>
<li> Model Training</li>
<li> Performance Testing: Use R2 and RMS From HW 5</li>
</ol></h3>

# Visuallization (HW4) <hr>

## Thing's to do <br>
<h3><ol>
<li>Visualize the dimention reduction data</li>
<li>Visualize the regression line with data point</li>
</ol></h3>